In [1]:
import os
import time
import glob
import shutil
import subprocess
import numpy as np
from PIL import Image
from predictor import COCODemo
from random import shuffle
from maskrcnn_benchmark.config import cfg
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

In [2]:
# this makes our figures bigger
pylab.rcParams['figure.figsize'] = 20, 12

In [3]:
# specify paths
path_to_videos = "/home/SharedFolder/CurrentDatasets/bdd100k_video_samples"

In [4]:
# confidence threshold
conf_thresh = 0.7

In [ ]:
# suffix for output file name
outfile_suffix = "_train_A_over_iter_final_ct" + str(conf_thresh)

In [ ]:
# specify config file
config_file = "/home/SharedFolder/git/csoehnel/maskrcnn-benchmark/configs/retinanet/retinanet_R-50-FPN_1x_finetune_nightdrive.yaml"
cfg.merge_from_file(config_file)

In [ ]:
# specify model-weight file
model_weight = "/home/SharedFolder/trained_models/night-drive/detector/20190309_RetinaNet_train_A_over_final_epoch/model_final.pth"
cfg.merge_from_list(["MODEL.WEIGHT", model_weight])

In [ ]:
cfg.merge_from_list(["MODEL.DEVICE", "cuda"])
coco_demo = COCODemo(
    cfg,
    min_image_size = 800,
    confidence_threshold = conf_thresh,
)

In [ ]:
videos = glob.glob(path_to_videos + "/*.mov")
shuffle(videos)

In [ ]:
def process_video(video):
    # (re-) create directories for extracted frames and target video
    file_name = os.path.basename(video)
    path_name = os.path.dirname(video)
    temp_path = os.path.join(path_name, file_name.split(".")[0])
    temp_pred_path = os.path.join(temp_path, "prediction")
    target_path = os.path.join(path_name, "demovideos")
    target_file = os.path.join(target_path,file_name.split(".mov")[0] + outfile_suffix + ".mp4")
    if os.path.isdir(temp_path):
        shutil.rmtree(temp_path)
    os.makedirs(temp_path)
    os.makedirs(temp_pred_path)
    if not os.path.isdir(target_path):
        os.makedirs(target_path)
    elif os.path.exists(target_file):
        # do nothing if file already processed
        return 0
    # extract frames from video
    bash_cmd = ["ffmpeg", "-i", video, "-start_number", "0", "-qscale:v", "2", temp_path + "/frame-%d.jpg"]
    subprocess.call(bash_cmd)
    # process frames
    frames = glob.glob(temp_path + "/*.jpg")
    for frame in frames:    
        pil_image = Image.open(frame).convert("RGB")
        # convert to BGR format
        image = np.array(pil_image)[:, :, [2, 1, 0]]
        # predict
        prediction = coco_demo.run_on_opencv_image(image, line_width = 3)
        # convert to RGB format
        pil_prediction = Image.fromarray(prediction[:, :, [2, 1, 0]])
        pil_prediction.save(os.path.join(temp_pred_path, os.path.basename(frame)), quality = 95)
    # construct video
    bash_cmd = ["ffmpeg", "-r", "30", "-f", "image2", "-i", temp_pred_path + "/frame-%d.jpg", "-vcodec", "libx264", "-crf", "18", target_file]
    subprocess.call(bash_cmd)
    # clean-up
    shutil.rmtree(temp_path)
    return len(frames)

In [ ]:
for i in range(len(videos)):
    print(f"Processing video {i + 1} of {len(videos)}", end = "")
    tic = time.time()
    n_frames = process_video(videos[i])
    toc = time.time()
    if n_frames > 0:
        print(f"... done in {toc - tic:.2f}s ({((toc - tic) / n_frames):.2f}s / frame)")
    else:
        print(f"... skipped. File exists.")

Processing video 1 of 1000... done in 156.65s (0.26s / frame)
Processing video 2 of 1000... done in 316.55s (0.26s / frame)
Processing video 3 of 1000... done in 325.05s (0.27s / frame)
Processing video 4 of 1000... done in 307.94s (0.26s / frame)
Processing video 5 of 1000... done in 319.04s (0.26s / frame)
Processing video 6 of 1000... done in 329.09s (0.27s / frame)
Processing video 7 of 1000... done in 340.26s (0.28s / frame)
Processing video 8 of 1000... done in 661.44s (0.28s / frame)
Processing video 9 of 1000... done in 315.04s (0.26s / frame)
Processing video 10 of 1000... done in 323.79s (0.27s / frame)
Processing video 11 of 1000... done in 318.98s (0.27s / frame)
Processing video 12 of 1000... done in 316.14s (0.26s / frame)
Processing video 13 of 1000